# Can You Forecast Daily New Covid Cases in Texas?

Billy Nayden

## The Datasets
* The data sets include daily new covid case count curves for TX, NY, and NJ:
 * **TX_Corona_Curve.csv** - 06/19/2020 - 07/06/2020, 18 days data, incomplete curve.
 * **NY_Corona_Curve.csv** - 03/20/2020 - 05/05/2020, 48 days data, completed curve.
 * **NJ_Corona_Curve.csv** - 03/26/2020 - 05/18/2020, 54 days data, completed curve.
 * **Corona_Curves_TX_NY_NJ.csv** - All the data above combined and normalized using each state's population. California and Florida were added to the data set from 6/19/2020 - 7/6/2020, as imcomplete curves will serve as a proxy for Texas Training data to predict from a 0 curve day input value from the Texas Curve. 
 
* This data was painfully collected from the daily new case counts graphs at: 
 * https://www.worldometers.info/coronavirus/usa/florida/
 * https://www.worldometers.info/coronavirus/usa/california/
 * https://www.worldometers.info/coronavirus/usa/texas/
 * https://www.worldometers.info/coronavirus/usa/new-york/
 * https://www.worldometers.info/coronavirus/usa/new-jersey/

## Data Definitions
* **Pop_Pct** - Represents the % of each respective state's population for the daily new case count (daily new cases / state population)
* **Curve_Day** - Curve day 1 starts when the daily new case count rises above a 0.00015 case to state population ratio.  The curve finishes when the daily new case count falls below a 0.00015 case to state population ratio.    
* **Cases** - The number of daily new Covid Cases for each state. The raw counts are only in the state level files. 
* **Pct_Change** - The change in this percentage for the prior day. (Pop_Pct - Previous day's Pop_Pct) 
* **M1 thru M7** - The prior value minus 1 to minus 7 days ago. 
* **Three_Day_Avg** - The 3 day average including today's value.
* **Seven_Day_Avg** - The 7 day average including today's value.

## NOTE!
**This notebook is not intended to show you the best way to forecast the curve, but rather to introduce you to a few tools that may help you do so.**

# HELPER FUNCTIONS 
## Importing the helper functions for scoring and for cv cross validation 
> Adapted from https://github.com/jakemdrew/CoronaCurves

In [1]:
#Use mean absolute error (MAE) to score the regression models created 
#(the scale of MAE is identical to the response variable)
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

#Function for Mean Absolute Percentage Error (MAPE) - Untested
#Adapted from - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python
def mape(y_actual, y_predicted): 
    mask = y_actual != 0
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100

#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
mape_scorer = make_scorer(score_func=mape, greater_is_better=False)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
                'MAPE': mape_scorer
               } 

In [2]:
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

    #print mean test_MAPE for all folds
    scores['test_MAPE'] = scores['test_MAPE']
    mape_avg = scores['test_MAPE'].mean()
    print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
    print(print_str.format(mape_avg=mape_avg))

    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    return scoresResults

Lets see if we can predict the Texas Corona Curves for the month of July.
> Using data from https://github.com/jakemdrew/CoronaCurves

In [3]:
import pandas as pd
import numpy as np
corona_data_url = 'https://raw.githubusercontent.com/andrewmejia600/CoronaCurves/master/Corona_Curves_TX_NY_NJ.csv'
corona_data = pd.read_csv(corona_data_url,low_memory=False)
corona_data["Date"] = pd.to_datetime(corona_data["Date"])
corona_data.sort_values(by=['Date'], inplace=True)
corona_data = corona_data[corona_data['State'] != 'TX'].copy()
corona_data = corona_data.reset_index()

y_regr = corona_data['Daily_New_Cases']
X_regr = corona_data.drop(['index','State','Date','Pop_Pct','Daily_New_Cases','Pct_Change', 'Pop_Pct'], 1)

X_regr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Curve_Day              137 non-null    int64  
 1   Pop_Pct_M1             137 non-null    float64
 2   Pop_Pct_M2             137 non-null    float64
 3   Pop_Pct_M3             137 non-null    float64
 4   Pop_Pct_M4             137 non-null    float64
 5   Pop_Pct_M5             137 non-null    float64
 6   Pop_Pct_M6             137 non-null    float64
 7   Pop_Pct_M7             137 non-null    float64
 8   Three_Day_Avg_Pop_Pct  137 non-null    float64
 9   Seven_Day_Avg_Pop_Pct  137 non-null    float64
 10  Pct_Change_M1          137 non-null    float64
 11  Pct_Change_M2          137 non-null    float64
 12  Pct_Change_M3          137 non-null    float64
 13  Pct_Change_M4          137 non-null    float64
 14  Pct_Change_M5          137 non-null    float64
 15  Pct_Ch

In [4]:
corona_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index                  137 non-null    int64         
 1   State                  137 non-null    object        
 2   Date                   137 non-null    datetime64[ns]
 3   Daily_New_Cases        137 non-null    int64         
 4   Curve_Day              137 non-null    int64         
 5   Pop_Pct                137 non-null    float64       
 6   Pop_Pct_M1             137 non-null    float64       
 7   Pop_Pct_M2             137 non-null    float64       
 8   Pop_Pct_M3             137 non-null    float64       
 9   Pop_Pct_M4             137 non-null    float64       
 10  Pop_Pct_M5             137 non-null    float64       
 11  Pop_Pct_M6             137 non-null    float64       
 12  Pop_Pct_M7             137 non-null    float64       
 13  Three

In [5]:
corona_data.head(n = 137)

,index,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
0,55,NY,2020-03-20,3052,1,0.000162,0.000124,0.000071,0.000039,0.000012,...,0.000038,0.000053,0.000032,0.000027,0.000001,0.000006,7.450000e-07,0.000000e+00,4.000000e-05,2.000000e-05
1,56,NY,2020-03-21,1993,2,0.000106,0.000162,0.000124,0.000071,0.000039,...,-0.000056,0.000038,0.000053,0.000032,0.000027,0.000001,5.584000e-06,7.450000e-07,1.000000e-05,1.000000e-05
2,57,NY,2020-03-22,5440,3,0.000289,0.000106,0.000162,0.000124,0.000071,...,0.000183,-0.000056,0.000038,0.000053,0.000032,0.000027,1.223000e-06,5.584000e-06,5.000000e-05,4.000000e-05
3,58,NY,2020-03-23,5123,4,0.000272,0.000289,0.000106,0.000162,0.000124,...,-0.000017,0.000183,-0.000056,0.000038,0.000053,0.000032,2.696200e-05,1.223000e-06,4.000000e-05,4.000000e-05
4,59,NY,2020-03-24,5516,5,0.000293,0.000272,0.000289,0.000106,0.000162,...,0.000021,-0.000017,0.000183,-0.000056,0.000038,0.000053,3.190800e-05,2.696200e-05,6.000000e-05,4.000000e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,15,FL,2020-07-04,11458,16,0.000535,0.000443,0.000472,0.000307,0.000285,...,0.000092,-0.000029,0.000166,0.000022,0.000039,-0.000153,-4.929907e-05,3.004673e-05,5.288162e-05,3.644860e-06
133,34,CA,2020-07-05,6027,17,0.000153,0.000212,0.000216,0.000239,0.000196,...,-0.000059,-0.000004,-0.000023,0.000042,-0.000005,0.000026,4.210127e-05,4.210127e-05,5.012658e-06,1.719711e-05
134,16,FL,2020-07-05,10059,17,0.000470,0.000535,0.000443,0.000472,0.000307,...,-0.000065,0.000092,-0.000029,0.000166,0.000022,0.000039,-1.525234e-04,-4.929907e-05,7.624611e-05,1.250334e-05
135,17,FL,2020-07-06,6336,18,0.000296,0.000470,0.000535,0.000443,0.000472,...,-0.000174,-0.000065,0.000092,-0.000029,0.000166,0.000022,3.864486e-05,-1.525234e-04,-7.788162e-07,1.020694e-05


Using a timeseries 10 CV split to train and test the models to elminate a bad fold due to chance.

In [6]:
#https://github.com/jakemdrew/CoronaCurves

from sklearn.model_selection import TimeSeriesSplit

n=10
cv = TimeSeriesSplit(n_splits=n)



Using a scaler object to transform the dynamic ranges of the data, the standard Scaler produced the lower MAE, but only after applying the target transformer.

In [7]:
#https://github.com/jakemdrew/CoronaCurves



from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PowerTransformer


#Fit to data for scaling

scaler = StandardScaler()

#scaler = PowerTransformer()

scaler.fit(X_regr)

# Transform training data

X_regr_scaled = scaler.transform(X_regr)

X_regr_scaled.shape

(137, 19)

Using a MLP regressor with differing activation  and solver functions to perform a grid search and return the best hyper parameters. Even though it is recommended to train on Scaled Data, I will not be scaling around the mean due to some of the curve estimates will give a infinty number when there is a mean of 0, ratherI will use the standard scaling after during my target transformation. 

> https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

> https://github.com/jakemdrew/CoronaCurves

In [8]:
#Create a Linear Regression object and perform a grid search to find the best parameters
from sklearn.neural_network import MLPRegressor

linreg = MLPRegressor(random_state=1, max_iter=10000)
parameters = { 'activation' : ['logistic', 'tanh', 'relu']
              ,'solver' : ['sgd', 'adam']
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , n_jobs=-1 
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_regr, y_regr)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.7min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=10000, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=1, shuffle=True, solver='adam',
                                    tol=0.0001, validation_fraction=0.1,
                                    verbose=False, warm_start=False),
             iid='deprecated', n_jobs=-1,
             pa

In [9]:
#Print the parameterization of the best estimator
regGridSearch.best_estimator_

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

We see the best MAE produced is 2517 number of cases. 

> https://github.com/jakemdrew/CoronaCurves

In [10]:

regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
#pd.set_option('display.float_format', '{:.17f}'.format)
pd.reset_option('display.float_format')
EvaluateRegressionEstimator(regEstimator, X_regr, y_regr, cv)

The average MAE for all cv folds is: 			 2517.4
The average MAE percentage (MAPE) for all cv folds is: 	 75.394
The average RMSE for all cv folds is: 			 2817.7
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,3040.416667,49.650810,3552.745069
1,3093.813756,54.082638,3363.903321
2,2525.592142,59.073213,2843.175287
3,1634.298702,49.729093,1916.069966
4,2313.742400,61.993723,2680.492566
5,2306.483021,98.872144,2505.010775
6,3581.782754,287.604209,3607.786441
7,995.117309,24.908495,1100.596383
8,2079.183002,27.560322,2638.407445
9,3603.285367,40.466485,3969.202826


Now lets apply a transormation of the quantiles on the target to better predict the curve of the data using 10 quantiles using the `TRANSFORMEDTARGETREGRESSOR` function to fit the curve

In [11]:
#https://scikit-learn.org/dev/auto_examples/compose/plot_transformed_target.html#sphx-glr-auto-examples-compose-plot-transformed-target-py
#https://github.com/jakemdrew/CoronaCurves

from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

MLP_reg = MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)


tt = TransformedTargetRegressor(regressor=MLP_reg, transformer=QuantileTransformer(n_quantiles=10,
                                    output_distribution='normal'))

We see an improvement in CV MAE by using the transformation on the target function, CV mean MAE is now 2068

> https://github.com/jakemdrew/CoronaCurves

In [12]:
EvaluateRegressionEstimator(tt, X_regr_scaled, y_regr, cv)

The average MAE for all cv folds is: 			 2068.0
The average MAE percentage (MAPE) for all cv folds is: 	 46.099
The average RMSE for all cv folds is: 			 2464.4
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,2520.324811,52.618350,3150.524120
1,1135.673511,25.833409,1730.483751
2,1480.790337,25.429100,1981.001727
3,2899.183040,71.295522,3432.247115
4,1315.171330,35.416473,1697.575272
5,1193.676262,39.843400,1398.656221
6,751.636525,57.988854,760.933173
7,2848.445901,63.396988,3091.880860
8,3106.776374,47.964841,3365.165278
9,3428.451749,41.199883,4036.016643


Let's import the data for running the simulation, there is data from FL and CA from 6-19 to 7-6 to serve as a proxy to TX since the curves look roughly the same. 

In [13]:
#https://github.com/jakemdrew/CoronaCurves

#import required Libraries
import pandas as pd
import numpy as np

# Load the corona curves dataset
corona_data = pd.read_csv(corona_data_url,low_memory=False)
corona_data["Date"] = pd.to_datetime(corona_data["Date"])
corona_data.sort_values(by=['Date'], inplace=True)

# Create the TX curve dataset 
# (we add the simulation data to this later)
tx_curve = corona_data[corona_data['State'] == 'TX'].copy()

print('Starting Training Dataset (rows, columns)')
print('----------------------------------------------------------')
corona_data.info(verbose=False)
print(' ')
print('Starting Texas Dataset')
print('----------------------------------------------------------')
tx_curve.info(verbose=False)

Starting Training Dataset (rows, columns)
----------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 55 to 54
Columns: 24 entries, State to Seven_Day_Avg_Pct_Chg
dtypes: datetime64[ns](1), float64(20), int64(2), object(1)
memory usage: 30.5+ KB
 
Starting Texas Dataset
----------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 36 to 54
Columns: 24 entries, State to Seven_Day_Avg_Pct_Chg
dtypes: datetime64[ns](1), float64(20), int64(2), object(1)
memory usage: 3.7+ KB


In [14]:
corona_data

,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,Pop_Pct_M5,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
55,NY,2020-03-20,3052,1,0.000162,0.000124,0.000071,0.000039,0.000012,0.000011,...,0.000038,0.000053,0.000032,0.000027,0.000001,0.000006,7.450000e-07,0.000000e+00,4.000000e-05,2.000000e-05
56,NY,2020-03-21,1993,2,0.000106,0.000162,0.000124,0.000071,0.000039,0.000012,...,-0.000056,0.000038,0.000053,0.000032,0.000027,0.000001,5.584000e-06,7.450000e-07,1.000000e-05,1.000000e-05
57,NY,2020-03-22,5440,3,0.000289,0.000106,0.000162,0.000124,0.000071,0.000039,...,0.000183,-0.000056,0.000038,0.000053,0.000032,0.000027,1.223000e-06,5.584000e-06,5.000000e-05,4.000000e-05
58,NY,2020-03-23,5123,4,0.000272,0.000289,0.000106,0.000162,0.000124,0.000071,...,-0.000017,0.000183,-0.000056,0.000038,0.000053,0.000032,2.696200e-05,1.223000e-06,4.000000e-05,4.000000e-05
59,NY,2020-03-24,5516,5,0.000293,0.000272,0.000289,0.000106,0.000162,0.000124,...,0.000021,-0.000017,0.000183,-0.000056,0.000038,0.000053,3.190800e-05,2.696200e-05,6.000000e-05,4.000000e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,CA,2020-07-05,6027,17,0.000153,0.000212,0.000216,0.000239,0.000196,0.000201,...,-0.000059,-0.000004,-0.000023,0.000042,-0.000005,0.000026,4.210127e-05,4.210127e-05,5.012658e-06,1.719711e-05
16,FL,2020-07-05,10059,17,0.000470,0.000535,0.000443,0.000472,0.000307,0.000285,...,-0.000065,0.000092,-0.000029,0.000166,0.000022,0.000039,-1.525234e-04,-4.929907e-05,7.624611e-05,1.250334e-05
17,FL,2020-07-06,6336,18,0.000296,0.000470,0.000535,0.000443,0.000472,0.000307,...,-0.000174,-0.000065,0.000092,-0.000029,0.000166,0.000022,3.864486e-05,-1.525234e-04,-7.788162e-07,1.020694e-05
35,CA,2020-07-06,6891,18,0.000174,0.000153,0.000212,0.000216,0.000239,0.000196,...,0.000022,-0.000059,-0.000004,-0.000023,0.000042,-0.000005,2.564557e-05,2.564557e-05,-2.879325e-05,4.122966e-07


We are removing the TX data completely from our training set and only using NY, NJ, FL, and CA.

In [15]:
corona_data = corona_data[(corona_data['State'] != 'TX')].copy()

In [16]:
corona_data 

,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,Pop_Pct_M5,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
55,NY,2020-03-20,3052,1,0.000162,0.000124,0.000071,0.000039,0.000012,0.000011,...,0.000038,0.000053,0.000032,0.000027,0.000001,0.000006,7.450000e-07,0.000000e+00,4.000000e-05,2.000000e-05
56,NY,2020-03-21,1993,2,0.000106,0.000162,0.000124,0.000071,0.000039,0.000012,...,-0.000056,0.000038,0.000053,0.000032,0.000027,0.000001,5.584000e-06,7.450000e-07,1.000000e-05,1.000000e-05
57,NY,2020-03-22,5440,3,0.000289,0.000106,0.000162,0.000124,0.000071,0.000039,...,0.000183,-0.000056,0.000038,0.000053,0.000032,0.000027,1.223000e-06,5.584000e-06,5.000000e-05,4.000000e-05
58,NY,2020-03-23,5123,4,0.000272,0.000289,0.000106,0.000162,0.000124,0.000071,...,-0.000017,0.000183,-0.000056,0.000038,0.000053,0.000032,2.696200e-05,1.223000e-06,4.000000e-05,4.000000e-05
59,NY,2020-03-24,5516,5,0.000293,0.000272,0.000289,0.000106,0.000162,0.000124,...,0.000021,-0.000017,0.000183,-0.000056,0.000038,0.000053,3.190800e-05,2.696200e-05,6.000000e-05,4.000000e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,FL,2020-07-04,11458,16,0.000535,0.000443,0.000472,0.000307,0.000285,0.000246,...,0.000092,-0.000029,0.000166,0.000022,0.000039,-0.000153,-4.929907e-05,3.004673e-05,5.288162e-05,3.644860e-06
34,CA,2020-07-05,6027,17,0.000153,0.000212,0.000216,0.000239,0.000196,0.000201,...,-0.000059,-0.000004,-0.000023,0.000042,-0.000005,0.000026,4.210127e-05,4.210127e-05,5.012658e-06,1.719711e-05
16,FL,2020-07-05,10059,17,0.000470,0.000535,0.000443,0.000472,0.000307,0.000285,...,-0.000065,0.000092,-0.000029,0.000166,0.000022,0.000039,-1.525234e-04,-4.929907e-05,7.624611e-05,1.250334e-05
17,FL,2020-07-06,6336,18,0.000296,0.000470,0.000535,0.000443,0.000472,0.000307,...,-0.000174,-0.000065,0.000092,-0.000029,0.000166,0.000022,3.864486e-05,-1.525234e-04,-7.788162e-07,1.020694e-05


We see the TX curve with prepopulated data.

In [17]:
tx_curve

,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,Pop_Pct_M5,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
36,TX,2020-06-18,3357,0,0.000112,0.000117,0.000148,0.000072,0.000045,0.000076,...,-0.000005,-0.000030,0.000076,0.000027,-0.000031,0.000003,0.000005,-0.000014,1.358974e-05,0.000006
37,TX,2020-06-19,4497,1,0.000150,0.000110,0.000120,0.000150,0.000070,0.000040,...,0.000038,-0.000005,-0.000030,0.000076,0.000027,-0.000031,0.000003,0.000005,9.400000e-07,0.000011
38,TX,2020-06-20,4250,2,0.000142,0.000150,0.000110,0.000120,0.000150,0.000070,...,-0.000008,0.000038,-0.000005,-0.000030,0.000076,0.000027,-0.000031,0.000003,8.240000e-06,0.000010
39,TX,2020-06-21,3125,3,0.000104,0.000142,0.000150,0.000110,0.000120,0.000150,...,-0.000038,-0.000008,0.000038,-0.000005,-0.000030,0.000076,0.000027,-0.000031,-2.590000e-06,0.000009
40,TX,2020-06-22,5112,4,0.000171,0.000104,0.000142,0.000150,0.000110,0.000120,...,0.000066,-0.000038,-0.000008,0.000038,-0.000005,-0.000030,0.000076,0.000027,6.860000e-06,0.000014
41,TX,2020-06-23,5370,5,0.000180,0.000171,0.000104,0.000142,0.000150,0.000110,...,0.000009,0.000066,-0.000038,-0.000008,0.000038,-0.000005,-0.000030,0.000076,1.249000e-05,0.000005
42,TX,2020-06-24,6177,6,0.000207,0.000180,0.000171,0.000104,0.000142,0.000150,...,0.000027,0.000009,0.000066,-0.000038,-0.000008,0.000038,-0.000005,-0.000030,3.402000e-05,0.000013
43,TX,2020-06-25,5960,7,0.000199,0.000207,0.000180,0.000171,0.000104,0.000142,...,-0.000007,0.000027,0.000009,0.000066,-0.000038,-0.000008,0.000038,-0.000005,9.450000e-06,0.000012
44,TX,2020-06-26,5614,8,0.000188,0.000199,0.000207,0.000180,0.000171,0.000104,...,-0.000012,-0.000007,0.000027,0.000009,0.000066,-0.000038,-0.000008,0.000038,2.720000e-06,0.000005
45,TX,2020-06-27,6079,9,0.000203,0.000188,0.000199,0.000207,0.000180,0.000171,...,0.000016,-0.000012,-0.000007,0.000027,0.000009,0.000066,-0.000038,-0.000008,-1.090000e-06,0.000009


We will initialize the  data set on the 18th of June to predict our curve from June 19 to August 7

In [18]:
tx_curve = tx_curve.iloc[[0]]
tx_curve

,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,Pop_Pct_M5,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
36,TX,2020-06-18,3357,0,0.000112,0.000117,0.000148,0.000072,0.000045,0.000076,...,-0.000005,-0.00003,0.000076,0.000027,-0.000031,0.000003,0.000005,-0.000014,0.000014,0.000006


Using the target transformation and MLP best hyper parameters, lets run the simulation to predict the number of cases not the proportion from June 19 to August 7.

In [19]:
#https://github.com/jakemdrew/CoronaCurves
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, quantile_transform
import datetime

# Retrain the model using the prior day's synthetic data


MLP_reg = MLP_reg = MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=10000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='sgd',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)








tt = TransformedTargetRegressor(regressor=MLP_reg, transformer=QuantileTransformer(n_quantiles=10,
                                    output_distribution='normal'))





Texas_Population = 29900000

#18 to 55 originally

for curve_day in range(0,50):
    # create the training dataset using the most recent synthetic data for each iteration
    y_syn = corona_data['Daily_New_Cases']
    X_syn = corona_data.drop(['State','Date','Pop_Pct','Daily_New_Cases','Pct_Change','Pop_Pct' ], 1)
    
    scaler = StandardScaler()
  
    
    X_syn_scaled = scaler.fit_transform(X_syn)
    
    #Fit model using Transformed Target Regressor 
    tt.fit(X_syn_scaled, y_syn)
    
    # Get the most recent curve day from the dataset
    prev_day = tx_curve[tx_curve['Curve_Day'] == curve_day].copy()
    
    # Now make the synthetic data required to get a prediction from the model for the next day  
    cur_day = tx_curve[tx_curve['Curve_Day'] == curve_day].copy()
    
    # Create the synthetic data for the curve day
    cur_day['Curve_Day'] = cur_day['Curve_Day'] + 1
    
    # Create the synthetic data for Pop_Pct
    cur_day['Pop_Pct_M7'] = cur_day['Pop_Pct_M6']
    cur_day['Pop_Pct_M6'] = cur_day['Pop_Pct_M5']
    cur_day['Pop_Pct_M5'] = cur_day['Pop_Pct_M4']
    cur_day['Pop_Pct_M4'] = cur_day['Pop_Pct_M3']
    cur_day['Pop_Pct_M3'] = cur_day['Pop_Pct_M2']
    cur_day['Pop_Pct_M2'] = cur_day['Pop_Pct_M1']
    cur_day['Pop_Pct_M1'] = cur_day['Pop_Pct']
    
    # Create the synthetic data for Pop_Pct averages
    cur_day['Three_Day_Avg_Pop_Pct'] = cur_day[['Pop_Pct_M1','Pop_Pct_M2','Pop_Pct_M3']].mean(axis=1) 
    cur_day['Seven_Day_Avg_Pop_Pct'] = cur_day[['Pop_Pct_M1','Pop_Pct_M2','Pop_Pct_M3',
                                                'Pop_Pct_M4','Pop_Pct_M5','Pop_Pct_M6','Pop_Pct_M7']].mean(axis=1) 
    
    # Create the synthetic data for Pct_Change
    cur_day['Pct_Change_M7'] = cur_day['Pct_Change_M6']
    cur_day['Pct_Change_M6'] = cur_day['Pct_Change_M5']
    cur_day['Pct_Change_M5'] = cur_day['Pct_Change_M4']
    cur_day['Pct_Change_M4'] = cur_day['Pct_Change_M3']
    cur_day['Pct_Change_M3'] = cur_day['Pct_Change_M2']
    cur_day['Pct_Change_M2'] = cur_day['Pct_Change_M1']
    cur_day['Pct_Change_M1'] = cur_day['Pct_Change']
    
    # Create the synthetic data for Pct_Change averages
    cur_day['Three_Day_Avg_Pct_Chg'] = cur_day[['Pct_Change_M1','Pct_Change_M2','Pct_Change_M3']].mean(axis=1) 
    cur_day['Seven_Day_Avg_Pct_Chg'] = cur_day[['Pct_Change_M1','Pct_Change_M2','Pct_Change_M3',
                                                'Pct_Change_M4','Pct_Change_M5','Pct_Change_M6','Pct_Change_M7']].mean(axis=1)
    
    #Make cur_day_X 
    cur_day_X = cur_day.drop(['State','Date','Pop_Pct','Daily_New_Cases','Pct_Change'], 1)
    cur_day_X_scaled = scaler.transform(cur_day_X)
    
    
    
    
    #Get the predicted Daily New Cases using the model
    cur_day_pop = tt.predict(cur_day_X_scaled)
    

    
    #Finish creating the data for this day's record
    cur_day['Date'] = cur_day['Date'] + datetime.timedelta(days=1)
    cur_day['Pop_Pct'] = cur_day_pop/Texas_Population
    cur_day['Pct_Change'] = cur_day['Pop_Pct'] - prev_day['Pct_Change'] 
    cur_day['Daily_New_Cases'] = np.rint(cur_day['Pop_Pct'] * Texas_Population).astype(int)
    
    
    
    
    
    # Append the current day's predicted data to the Texas curve simulation output
    tx_curve = tx_curve.append(cur_day)
    # Append the current day's predicted data to our training dataset as a feedback loop
    corona_data = corona_data.append(cur_day)

In [20]:
pd.set_option('display.max_columns', 500)
tx_curve['NN_PRED_OUT'] = tx_curve['Daily_New_Cases']
tx_curve['Actual_Daily_New_Cases'] = tx_curve['Daily_New_Cases']

In [21]:
tx_curve = tx_curve.drop('Daily_New_Cases', axis = 1)

In [22]:
tx_curve = tx_curve.reset_index()

In [23]:
tx_curve = tx_curve.drop('index', axis = 1)

Filling in Actual Daily New Texas Cases

In [24]:
tx_curve.at[1, "Actual_Daily_New_Cases"] = 4497
tx_curve.at[2, "Actual_Daily_New_Cases"] = 4250
tx_curve.at[3, "Actual_Daily_New_Cases"] = 3125
tx_curve.at[4, "Actual_Daily_New_Cases"] = 5112
tx_curve.at[5, "Actual_Daily_New_Cases"] = 5370
tx_curve.at[6, "Actual_Daily_New_Cases"] = 6177
tx_curve.at[7, "Actual_Daily_New_Cases"] = 5960
tx_curve.at[8, "Actual_Daily_New_Cases"] = 5614
tx_curve.at[9, "Actual_Daily_New_Cases"] = 6079
tx_curve.at[10, "Actual_Daily_New_Cases"] = 4330
tx_curve.at[11, "Actual_Daily_New_Cases"] = 6135
tx_curve.at[12, "Actual_Daily_New_Cases"] = 7959
tx_curve.at[13, "Actual_Daily_New_Cases"] = 8240
tx_curve.at[14, "Actual_Daily_New_Cases"] = 7734
tx_curve.at[15, "Actual_Daily_New_Cases"] = 8052
tx_curve.at[16, "Actual_Daily_New_Cases"] = 5563
tx_curve.at[17, "Actual_Daily_New_Cases"] = 5288
tx_curve.at[18, "Actual_Daily_New_Cases"] = 9241
tx_curve.at[19, "Actual_Daily_New_Cases"] = 9828
tx_curve.at[20, "Actual_Daily_New_Cases"] = 10649
tx_curve.at[21, "Actual_Daily_New_Cases"] = 11901
tx_curve.at[22, "Actual_Daily_New_Cases"] = 10554
tx_curve.at[23, "Actual_Daily_New_Cases"] = 8674
tx_curve.at[24, "Actual_Daily_New_Cases"] = 6363
tx_curve.at[25, "Actual_Daily_New_Cases"] = 9457
tx_curve.at[26, "Actual_Daily_New_Cases"] = 11514
tx_curve.at[27, "Actual_Daily_New_Cases"] = 12490
tx_curve.at[28, "Actual_Daily_New_Cases"] = 10243
tx_curve.at[29, "Actual_Daily_New_Cases"] = 9496
tx_curve.at[30, "Actual_Daily_New_Cases"] = 7945
tx_curve.at[31, "Actual_Daily_New_Cases"] = 8709
tx_curve.at[32, "Actual_Daily_New_Cases"] = 7925
tx_curve.at[33, "Actual_Daily_New_Cases"] = 10331
tx_curve.at[34, "Actual_Daily_New_Cases"] = 10528
tx_curve.at[35, "Actual_Daily_New_Cases"] = 9402
tx_curve.at[36, "Actual_Daily_New_Cases"] = 8552
tx_curve.at[37, "Actual_Daily_New_Cases"] = 7735
tx_curve.at[38, "Actual_Daily_New_Cases"] = 4309
tx_curve.at[39, "Actual_Daily_New_Cases"] = 6187
tx_curve.at[40, "Actual_Daily_New_Cases"] = 11037
tx_curve.at[41, "Actual_Daily_New_Cases"] = 9217
tx_curve.at[42, "Actual_Daily_New_Cases"] = 8843
tx_curve.at[43, "Actual_Daily_New_Cases"] = 9750
tx_curve.at[44, "Actual_Daily_New_Cases"] = 6720
tx_curve.at[45, "Actual_Daily_New_Cases"] = 5142
tx_curve.at[46, "Actual_Daily_New_Cases"] = 6997
tx_curve.at[47, "Actual_Daily_New_Cases"] = 11210
tx_curve.at[48, "Actual_Daily_New_Cases"] = 9625
tx_curve.at[49, "Actual_Daily_New_Cases"] = 7630
tx_curve.at[50, "Actual_Daily_New_Cases"] = 7675




In [25]:
test_mae_data_url = 'https://raw.githubusercontent.com/jakemdrew/CoronaCurves/master/Corona_MAE.csv'

test_mae_data = pd.read_csv(test_mae_data_url)

test_mae_data['TX Date'] = pd.to_datetime(test_mae_data['TX Date'])

In [26]:
test_mae_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TX Date        32 non-null     datetime64[ns]
 1   Curve_Day      32 non-null     int64         
 2   TX New Cases   32 non-null     int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 896.0 bytes


In [27]:
test_mae_data.columns

Index(['TX Date', 'Curve_Day', 'TX New Cases '], dtype='object')

Predicted vs Actual

In [28]:
tx_curve = tx_curve[['State', 'Date', 'Curve_Day','Actual_Daily_New_Cases', 'NN_PRED_OUT']].merge(test_mae_data, left_on = ['Date', 'Curve_Day'], right_on = ['TX Date', 'Curve_Day'], how = 'outer')

In [29]:
tx_curve[['State', 'Date', 'Curve_Day','Actual_Daily_New_Cases', 'TX New Cases ', 'NN_PRED_OUT']]

,State,Date,Curve_Day,Actual_Daily_New_Cases,TX New Cases,NN_PRED_OUT
0,TX,2020-06-18,0,3357,NaN,3357
1,TX,2020-06-19,1,4497,NaN,3908
2,TX,2020-06-20,2,4250,NaN,4717
3,TX,2020-06-21,3,3125,NaN,4523
4,TX,2020-06-22,4,5112,NaN,8462
5,TX,2020-06-23,5,5370,NaN,8592
6,TX,2020-06-24,6,6177,NaN,9444
7,TX,2020-06-25,7,5960,NaN,10868
8,TX,2020-06-26,8,5614,NaN,10521
9,TX,2020-06-27,9,6079,NaN,11033


In [30]:
import altair as alt

tx_curve_melt = tx_curve.melt(id_vars=['Date'], value_vars=['NN_PRED_OUT', 'Actual_Daily_New_Cases'], var_name='Pred_v_Act_Cases', value_name='NEW_CASES')

In [31]:
tx_curve_melt

,Date,Pred_v_Act_Cases,NEW_CASES
0,2020-06-18,NN_PRED_OUT,3357
1,2020-06-19,NN_PRED_OUT,3908
2,2020-06-20,NN_PRED_OUT,4717
3,2020-06-21,NN_PRED_OUT,4523
4,2020-06-22,NN_PRED_OUT,8462
...,...,...,...
97,2020-08-03,Actual_Daily_New_Cases,6997
98,2020-08-04,Actual_Daily_New_Cases,11210
99,2020-08-05,Actual_Daily_New_Cases,9625
100,2020-08-06,Actual_Daily_New_Cases,7630


CORONA CURVE PREDICTIONS VS ACTUAL REPORTED NEW CASES

In [32]:
alt.Chart(tx_curve_melt).mark_line().encode(
    x='Date:T',
    y='NEW_CASES:Q',
    color='Pred_v_Act_Cases',
    strokeDash='Pred_v_Act_Cases',
).properties(width = 750, height = 700).configure_axis(grid=False) 

alt.Chart(...)

Overall MAE for prediction of Texas Corona Curve for 7/7/2020 - 8/7/2020

In [36]:
overall_MAE_NN = sum(abs(tx_curve.NN_PRED_OUT[19:51] - tx_curve['TX New Cases '][19:51])) / (len(tx_curve[19:51]))

In [37]:
mae_Dict = {'NN_MAE': int(overall_MAE_NN)}
Overall_MAE = pd.DataFrame.from_dict(mae_Dict, orient='index', columns = ['Overall MAE'])

In [38]:
Overall_MAE

,Overall MAE
NN_MAE,1932
